In [30]:
import os

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resume.pdf")
documents = loader.load()

documents[:1]  # preview

c:\Users\KIIT0001\rag-resume\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2026-01-24T08:34:34+00:00', 'author': '', 'keywords': '', 'moddate': '2026-01-24T08:34:34+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Siddhartha Chakraborty\nComputer Science Undergraduate — Aspiring Backend Engineer\n♂phone9832688120 /envel⌢pex@x.com /linkedinlinkedin.com/in/siddhartha-chakraborty-680a58281 /githubgithub.com/SEEDART007 /gl⌢be\nseedart.netlify.app\nSummary\nComputer Science undergraduate with a strong focus on backend engineering and API development. Experienced in building\nsecure and scalable REST APIs, implementing authentication and authorization, integrating databases, and deploying\nproduction-ready applications. Comfortable working with Node.js, Express, dat

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)
len(chunks)


7

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_14248\3055314890.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 395.88it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [29]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings # Updated 2026 import
import os

# 1. Your existing setup
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Build the vectorstore in RAM
vectorstore = FAISS.from_documents(chunks, embeddings)

# 3. ADD THIS LINE: Save the RAM data to a physical folder
# This will create a folder named 'faiss_index' containing your 2 missing files
vectorstore.save_local("faiss_index")

print("Successfully created 'faiss_index' folder with index.faiss and index.pkl")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 255.44it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Successfully created 'faiss_index' folder with index.faiss and index.pkl


In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0
)


In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)


retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


response = rag_chain.invoke("Who is Siddhartha?")
print(response)

Siddhartha Chakraborty is a Computer Science undergraduate who aspires to be a Backend Engineer. He has experience in building secure and scalable REST APIs, implementing authentication and authorization, and deploying production-ready applications.
